In [104]:
!ls ../4000000to4999999_BlockTransaction

4000000to4999999_BlockTransaction.csv  processed.csv  time.csv
extracted_labels.csv		       readme.txt     tx_counts.csv


In [2]:
path = '../3000000to3999999_BlockTransaction/'

In [1]:
import pandas as pd
import torch
import networkx as nx
from matplotlib.pyplot import figure
from torch_geometric.data import Dataset, Data, DataLoader
from scipy.linalg import fractional_matrix_power
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from torch_geometric.data import InMemoryDataset
from torch_geometric.loader import DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
df_time = pd.read_csv(path+'time.csv')

In [4]:
df_tx_counts = pd.read_csv(path+'tx_counts.csv')
df_tx_counts

,0,1,tx_sent,tx_rec
0,isowallet,0xe76fe52a251c8f3a5dcd657e47a6c8d16fdf4bfa,51.0,6.0
1,mining,0xa42af2c70d316684e57aefcc6e393fecb1c7e84e,148831.0,6.0
2,mining,0x2a65aca4d5fc5b5c859090a6c34d164135398226,575304.0,3.0
3,mining,0xa027231f42c80ca4125b5cb962a21cd4f812e88f,10059.0,0.0
4,mining,0xea674fdde714fd979de3edf0f56aa9716b898ec8,1243235.0,6.0
...,...,...,...,...
74,mining,0x9d551f41fed6fc27b719777c224dfecce170004d,1.0,0.0
75,phish_hack,0x6f931bac260e7fcbaa0244d3b43a2bd9e9acf698,18.0,2.0
76,phish_hack,0xba83e9ce38b10522e3d6061a12779b7526839eda,2.0,90.0
77,phish_hack,0xc57f1148855e67763a694f7f2c0e68230adc686e,12.0,21.0


In [5]:
df_time

,0
0,2017-01-15
1,2017-02-19
2,2017-03-17
3,2017-04-03
4,2017-04-24
5,2017-05-10
6,2017-05-23
7,2017-06-01
8,2017-06-09
9,2017-06-15


In [6]:
df_tx_counts[df_tx_counts['tx_sent']>100]

,0,1,tx_sent,tx_rec
1,mining,0xa42af2c70d316684e57aefcc6e393fecb1c7e84e,148831.0,6.0
2,mining,0x2a65aca4d5fc5b5c859090a6c34d164135398226,575304.0,3.0
3,mining,0xa027231f42c80ca4125b5cb962a21cd4f812e88f,10059.0,0.0
4,mining,0xea674fdde714fd979de3edf0f56aa9716b898ec8,1243235.0,6.0
5,mining,0x4bb96091ee9d802ed039c4d1a5f6216f90f81b01,48711.0,6.0
6,mining,0xd34da389374caad1a048fbdc4569aae33fd5a375,257368.0,464.0
7,mining,0x52bc44d5378309ee2abf1539bf71de1b7d7be3b5,1064850.0,24.0
8,compromised,0x027beefcbad782faf69fad12dee97ed894c68549,11503.0,1160.0
9,compromised,0x0ee4e2d09aec35bdf08083b649033ac0a41aa75e,1919.0,33.0
10,gambling,0x777f415324d56e1d54fa832902d8797db7a4c57c,823.0,902.0


In [347]:
df_addr = df[df['to'] =='0x70faa28a6b8d6829a4b1e649d26ec9a2a39ba413']

In [348]:
df_addr

,timestamp,from,to,value,gasLimit,gasPrice,gasUsed
72835,1484696956,0x32be343b94f860124dc4fee278fdcbd38c102d88,0x70faa28a6b8d6829a4b1e649d26ec9a2a39ba413,1499990000000000000000,333333,30000000000,21000
214090,1485132938,0x32be343b94f860124dc4fee278fdcbd38c102d88,0x70faa28a6b8d6829a4b1e649d26ec9a2a39ba413,1198990000000000000000,333333,32015999998,21000
234719,1485190245,0xfbb1b73c4f0bda4f67dca266ce6ef42f520fbb98,0x70faa28a6b8d6829a4b1e649d26ec9a2a39ba413,1800000000000000000000,121000,20000000000,21000
455813,1485897616,0x1151314c646ce4e0efd76d1af4760ae66a9fe30f,0x70faa28a6b8d6829a4b1e649d26ec9a2a39ba413,5000000000000000000,90000,20000000000,21000
484526,1485992396,0x9e6316f44baeeee5d41a1070516cc5fa47baf227,0x70faa28a6b8d6829a4b1e649d26ec9a2a39ba413,80592000000000000000,31000,20000000000,21000
...,...,...,...,...,...,...,...
12846837,1499633458,0x7fd7d7890f17ca9ca0d6b669c86241d4c3e0bcc0,0x70faa28a6b8d6829a4b1e649d26ec9a2a39ba413,233071780000000000,21000,10000000000,21000
12846870,1499633514,0x3fd27a46be37da8d3ce7f545972391649637f603,0x70faa28a6b8d6829a4b1e649d26ec9a2a39ba413,19999580000000000000,21000,10000000000,21000
12846871,1499633514,0x968949d18fafc987d57eed8e5c60301e2ef6b994,0x70faa28a6b8d6829a4b1e649d26ec9a2a39ba413,8324912920000000000,21000,10000000000,21000
12846872,1499633514,0x5ef443705c5f7ca7f785ba0e23307f0548f307c6,0x70faa28a6b8d6829a4b1e649d26ec9a2a39ba413,999580000000000000,21000,10000000000,21000


In [126]:
print(date.fromtimestamp(df_addr['timestamp'].iloc[-1]))
print(date.fromtimestamp(df_addr['timestamp'].iloc[0]))


2017-07-09
2017-07-04


In [133]:
print(date.fromtimestamp(df_addr['timestamp'].iloc[0]))
print(date.fromtimestamp(df_addr['timestamp'].iloc[0]+700000))


2017-07-04
2017-07-12


In [7]:

df = pd.read_csv(path+'processed.csv')

In [8]:
df.shape

(12846971, 7)

In [9]:
df.shape

(12846971, 7)

In [10]:
df.head()

,timestamp,from,to,value,gasLimit,gasPrice,gasUsed
0,1484475035,0x4bb96091ee9d802ed039c4d1a5f6216f90f81b01,0x56c830805669f366a7b93411588954e1e1b2aab6,4950000000000000000,39000,20000000000,21000
1,1484475035,0xea674fdde714fd979de3edf0f56aa9716b898ec8,0xde088812a9c5005b0dc8447b37193c9e8b67a1ff,1011962068786510200,90000,20000000000,21000
2,1484475035,0xea674fdde714fd979de3edf0f56aa9716b898ec8,0xdeb5ebe6676ddddb280ea44f284e100edb9b9d4b,1011957331582025000,90000,20000000000,21000
3,1484475035,0xea674fdde714fd979de3edf0f56aa9716b898ec8,0xe7268aadb21f48a3b65f0880b6b9480217995979,1008362508265353200,90000,20000000000,21000
4,1484475035,0xea674fdde714fd979de3edf0f56aa9716b898ec8,0xf8c911c68f6a6b912fe735bbd953c3379336cbf3,1016119717874404400,90000,20000000000,21000


In [11]:
import torch
from torch_geometric.data import Data


In [61]:
def receiving_transaction_graph_h4_timed(node,st_time,end_time):
    
    timed_df = df[df['timestamp'].between(st_time,end_time)]
    
    neigh1 = timed_df[timed_df['to'] == node]
    
    neigh2 = timed_df[timed_df['to'].isin(neigh1['from'])]
    neigh2=neigh2[neigh2['to'] != node]
    
    neigh3 = timed_df[timed_df['to'].isin(neigh2['from'])]
    neigh3 = neigh3[~neigh3['to'].isin(neigh2['to'])]
    
    neigh4 = timed_df[timed_df['to'].isin(neigh3['from'])]
    neigh4 = neigh4[~neigh4['to'].isin(neigh3['to'])]
    
    return (neigh1,neigh2,neigh3,neigh4)
    
    

In [63]:
st_time = 1484475035
duration = 700000
end_time = st_time+duration
(neigh1,neigh2,neigh3,neigh4) = receiving_transaction_graph_h4_timed(addr,st_time,end_time)


(407, 7)


In [65]:
print(neigh4.shape)

(19033, 7)


In [55]:
def receiving_transaction_graph_h4(node):
    
    
    neigh1 = df[df['to'] == node]
    
    neigh2 = df[df['to'].isin(neigh1['from'])]
    neigh2=neigh2[neigh2['to'] != node]
    
    neigh3 = df[df['to'].isin(neigh2['from'])]
    neigh3 = neigh3[~neigh3['to'].isin(neigh2['to'])]
    
    neigh4 = df[df['to'].isin(neigh3['from'])]
    neigh4 = neigh4[~neigh4['to'].isin(neigh3['to'])]
    
    return (neigh1,neigh2,neigh3,neigh4)
    
    

In [56]:
addr = '0x96fc4553a00c117c5b0bed950dd625d1c16dc894'
(neigh1,neigh2,neigh3,neigh4) = receiving_transaction_graph_h4(addr)

In [60]:
neigh4.shape

(2347931, 7)

In [25]:
addr = '0x96fc4553a00c117c5b0bed950dd625d1c16dc894'
neigh1 = df[df['to'] == addr]
neigh1.shape

(50585, 7)

In [29]:
neigh2 = df[df['to'].isin(neigh1['from'])]

neigh2=neigh2[neigh2['to'] != addr]

neigh2.shape

(209648, 7)

In [45]:
neigh3 = df[df['to'].isin(neigh2['from'])]
neigh3 = neigh3[~neigh3['to'].isin(neigh2['to'])]
neigh3.shape

(1440182, 7)

In [54]:
neigh4 = df[df['to'].isin(neigh3['from'])]
neigh4 = neigh4[~neigh4['to'].isin(neigh3['to'])]
neigh4.shape

(2347931, 7)

In [46]:
n1_sr = pd.Series(neigh2.index)
n2_sr = pd.Series(neigh3.index)

In [47]:
n2_sr.isin(n1_sr).sum()

0

In [48]:
n2_sr[n2_sr.isin(n1_sr)]

Series([], dtype: int64)

In [53]:
print(neigh2.loc[21])
# print(neigh3.loc[21])

timestamp                                    1484475182
from         0xaa7abccc702638d94044bfc33c5fef9741c3cab3
to           0x7ed1e469fcb3ee19c0366d829e291451be638e59
value                                 53291050000000000
gasLimit                                          21000
gasPrice                                    40000000000
gasUsed                                           21000
Name: 21, dtype: object


In [12]:
addr = '0x96fc4553a00c117c5b0bed950dd625d1c16dc894'
neigh1 = df[df['from'] == addr]
neigh1.shape

(46548, 7)

In [316]:
neigh2 = df[df['from'].isin(neigh1['to'])]

neigh2=neigh2[neigh2['from'] != addr]
neigh2.shape

(178821, 7)

In [317]:
neigh3 = df[df['from'].isin(neigh2['to'])]
neigh3 = neigh3[~neigh3['from'].isin(neigh2['from'])]
neigh3.shape

(973989, 7)

In [318]:
neigh4 = df[df['from'].isin(neigh3['to'])]
neigh4 = neigh4[~neigh4['from'].isin(neigh3['from'])]
neigh4.shape

(3943959, 7)

In [325]:
neigh4['from'].unique().shape

(298835,)

In [326]:
def extract_graph_h4(node):
    neigh1 = df[df['from'] == node]
    
    neigh2 = df[df['from'].isin(neigh1['to'])]
    neigh2=neigh2[neigh2['from'] != node]
    
    neigh3 = df[df['from'].isin(neigh2['to'])]
    neigh3 = neigh3[~neigh3['from'].isin(neigh2['from'])]
    
    neigh4 = df[df['from'].isin(neigh3['to'])]
    neigh4 = neigh4[~neigh4['from'].isin(neigh3['from'])]
    
    return (neigh1,neigh2,neigh3,neigh4)
    
    


In [331]:
addr = '0xea674fdde714fd979de3edf0f56aa9716b898ec8'
(neigh1,neigh2,neigh3,neigh4) = extract_graph_h4(addr)

In [334]:
df['timestamp']

0           1484475035
1           1484475035
2           1484475035
3           1484475035
4           1484475035
               ...    
12846966    1499633540
12846967    1499633540
12846968    1499633540
12846969    1499633540
12846970    1499633540
Name: timestamp, Length: 12846971, dtype: int64

In [336]:
def extract_graph_h4_timed(node,st_time,end_time):
    
    timed_df = df[df['timestamp'].between(st_time,end_time)]
    
    neigh1 = timed_df[timed_df['from'] == node]
    
    neigh2 = timed_df[timed_df['from'].isin(neigh1['to'])]
    neigh2=neigh2[neigh2['from'] != node]
    
    neigh3 = timed_df[timed_df['from'].isin(neigh2['to'])]
    neigh3 = neigh3[~neigh3['from'].isin(neigh2['from'])]
    
    neigh4 = timed_df[timed_df['from'].isin(neigh3['to'])]
    neigh4 = neigh4[~neigh4['from'].isin(neigh3['from'])]
    
    return (neigh1,neigh2,neigh3,neigh4)
    
    


In [338]:
st_time = 1484475035
duration = 700000
end_time = st_time+duration
(neigh1,neigh2,neigh3,neigh4)=extract_graph_h4_timed(addr,st_time,end_time)

In [342]:
neigh4.shape

(6070, 7)

In [344]:
print(date.fromtimestamp(st_time))
print(date.fromtimestamp(end_time))

2017-01-15
2017-01-23


In [301]:
n1_sr = pd.Series(neigh3.index)
n2_sr = pd.Series(neigh4.index)

In [302]:
n2_sr.isin(n1_sr).sum()

0

In [175]:
class Graph:
    def __init__(self):
        self.nodes = []
        self.neighbors = []
        self.depth = 0
        self.hops = {}
    
    def add_node(self,node):
        self.nodes.append(node)
        self.add_neighbors(neighbors_from(node))
        
    def add_neighbors(self,neighbors):
        self.neighbors.append(neighbors)
        
    def expand_graph(self):
        current_len = len(self.nodes)
        for neigh in self.neighbors[0]:
            self.add_node(neigh)
        self.hops[self.depth] = (current_len,len(self.nodes)-1)
    def expand_graph2(self):
        current_len = len(self.nodes)
        for i in range(self.hops[self.depth][0],self.hops[self.depth][1]):
#             print(self.nodes[i])
            for neigh in self.neighbors[i]:
                self.add_node(neigh)
        self.depth +=1
        self.hops[self.depth] = (current_len,len(self.nodes))


In [176]:
graph = Graph()
addr = '0x21f74c6bbc1e3ab9f0205e12de3a9daa14351aed'
graph.add_node(addr)
graph.expand_graph()


In [187]:
graph.expand_graph2()

KeyboardInterrupt: 

In [178]:
graph.hops

{0: (1, 105), 1: (106, 433)}

In [180]:
len(graph.nodes)

433

In [186]:
graph.depth

1

In [183]:
graph.nodes[-2:]

['0xb36cbe7f95a39984384e6aa4068b02c1697ef80e',
 '0xf90c9ac616ecfefb3860aaa5bc33caf9bc606441']

In [181]:
for i in range(graph.hops[0][0],graph.hops[0][0]+graph.hops[0][1]):
            print(graph.nodes[i])

0x009eca640d78c7eaa234602a1485859db5d2784b
0x73e08d9b97a703fe07bf666efea1d79824abccf4
0x87e4aa5f513d9ac81b200d4db1ef5ec7e6939363
0xcdb43e4db5ebcdaffa5c3be90c7fc7979ac0644e
0xede08edab4c27f0edd9a7eb988626ab63a6f7c06
0x87b3c104a8a787cac7d2269b13bbf3ab68750c56
0x80c0a96b780a5cfcd8c8e71c71076c684e817d7c
0x561780c5002e94e9fe662c634df12a8ddc4a7069
0xcd518b4370738aa2a3c73a103258b5f4cc5eb37e
0x370f31dd796666330c9119305ef7a8efdeb04879
0xe014773e64c54cf87547fb01580d1b7b03b3bfb6
0x99722d376145463eca4e71c423a4dcc8e9dbda13
0x7e7795e5e3fd2a9e9911d2b35e3ccbfe2917ecb0
0x603f3fcd4c738c3f182f9379b448bde842370964
0x98faf4aaa59a55687a1f65b0940b83e505528af7
0x8477cd6ec6af9770b352a31cae45f73360c812e1
0x62db875fd1205c07ef42efaa78685c0e37f4245e
0xb762bf9932aa3d21c818b2b7067c7c112fcd99ca
0x43665332e1a586e78068ad4004d99bfcf5ff5608
0x1fd15ec8f3ab9a1f23362dfd2695e89e667d5382
0xae3349b9d387bf9e8b3836c04edd40a6abc4ec69
0x581e17c1f0f21d7bc1a76744f7c56dc50fcdee94
0x37a8036bdc566a471d19ddc5f3ebf9b01991ed3d
0x1ef08f4d6

In [182]:
for i in range(graph.hops[1][0],graph.hops[1][1]):
            print(graph.nodes[i])

0xd6d6b2b531e495167bcb3856e1d5a5133965eb6c
0xc8f35e1acdabd90bf790bd2e3b490806feea36cb
0x4405c29eba44bf5d5ab722fecaea96c44cec33a5
0x57dfbb9b08cd9759485d4a405a9054b213425f27
0xa988181cbfd86bbd1c354982dd40d02e44515b1c
0xc12baab11f7e19f3185f3ebe9ea07d85fe3347b2
0x6dd54e44243a1668d031e3a4f6f144ed878e3497
0xfaacab691143fa4cb7447f84c2e43cd08223a800
0x04d206b150016c425c1d5ea9a3baf86dc5df873b
0x14787d23c222d296b913475062c13b8f5dec0ef8
0xa9fac489dfad7565946f463deb1e5fb54b0af7e1
0xbec4b07fc71d1a82df908cea00ec0af74c7dfb53
0xee121ceb297e9e7d8c9e00510f104c6fec3ecdf4
0x6032d8d091634a554b17165e154262069c9f8fbf
0xb763df66250cad7585eb78592346b0439df85a90
0xa2b3a019a1c0bfcbcf5fcd5b0ae0c6c6637f6823
0x8bcc5fa47d93f78336ff22c9d0fe369a4937e0c7
0xbb8f520b9cb86e0cbe6b14ca4131eb96a1eef923
0x868c30716d374fb4d76039df1afb5ac835770f72
0x4bc6b6da3831630c6efc2fbad74dfa96e730bf76
0xf589ad43b31a692e8bd36b692f9140757c3b95fe
0xd83317db7e37cb2b8113895f8cdcded5548ff284
0x350b1455d532df8c293b2e6c0c8e7d41319bffd9
0x8f4b7a05a

In [92]:
graph = Graph()
addr = '0x21f74c6bbc1e3ab9f0205e12de3a9daa14351aed'
graph.nodes.append(addr)
graph.neighbors.append(neighbors_from(addr))

for neigh in graph.neighbors[0]:
    graph.nodes.append(neigh)
    graph.neighbors.append(neighbors_from(neigh))

In [80]:
def neighbors_from(addr):
    tx_from_addr = df[df['from'] == addr]
    neighbors = tx_from_addr['to'].unique()
    return neighbors
neighbors_from('0x21f74c6bbc1e3ab9f0205e12de3a9daa14351aed')

array(['0x009eca640d78c7eaa234602a1485859db5d2784b',
       '0x73e08d9b97a703fe07bf666efea1d79824abccf4',
       '0x87e4aa5f513d9ac81b200d4db1ef5ec7e6939363',
       '0xcdb43e4db5ebcdaffa5c3be90c7fc7979ac0644e',
       '0xede08edab4c27f0edd9a7eb988626ab63a6f7c06',
       '0x87b3c104a8a787cac7d2269b13bbf3ab68750c56',
       '0x80c0a96b780a5cfcd8c8e71c71076c684e817d7c',
       '0x561780c5002e94e9fe662c634df12a8ddc4a7069',
       '0xcd518b4370738aa2a3c73a103258b5f4cc5eb37e',
       '0x370f31dd796666330c9119305ef7a8efdeb04879',
       '0xe014773e64c54cf87547fb01580d1b7b03b3bfb6',
       '0x99722d376145463eca4e71c423a4dcc8e9dbda13',
       '0x7e7795e5e3fd2a9e9911d2b35e3ccbfe2917ecb0',
       '0x603f3fcd4c738c3f182f9379b448bde842370964',
       '0x98faf4aaa59a55687a1f65b0940b83e505528af7',
       '0x8477cd6ec6af9770b352a31cae45f73360c812e1',
       '0x62db875fd1205c07ef42efaa78685c0e37f4245e',
       '0xb762bf9932aa3d21c818b2b7067c7c112fcd99ca',
       '0x43665332e1a586e78068ad4004d99bfcf5ff

In [82]:
addr = '0x21f74c6bbc1e3ab9f0205e12de3a9daa14351aed'
neighbors = neighbors_from(addr)
graph = []
graph.append((addr,neighbors))
for neigh in neighbors:
    neighbors = neighbors_from(neigh)
    graph.append((neigh,neighbors))

In [85]:
for item in graph:
    print(item[0],item[1].shape)

0x21f74c6bbc1e3ab9f0205e12de3a9daa14351aed (105,)
0x009eca640d78c7eaa234602a1485859db5d2784b (0,)
0x73e08d9b97a703fe07bf666efea1d79824abccf4 (184,)
0x87e4aa5f513d9ac81b200d4db1ef5ec7e6939363 (2,)
0xcdb43e4db5ebcdaffa5c3be90c7fc7979ac0644e (1,)
0xede08edab4c27f0edd9a7eb988626ab63a6f7c06 (1,)
0x87b3c104a8a787cac7d2269b13bbf3ab68750c56 (0,)
0x80c0a96b780a5cfcd8c8e71c71076c684e817d7c (6,)
0x561780c5002e94e9fe662c634df12a8ddc4a7069 (0,)
0xcd518b4370738aa2a3c73a103258b5f4cc5eb37e (0,)
0x370f31dd796666330c9119305ef7a8efdeb04879 (0,)
0xe014773e64c54cf87547fb01580d1b7b03b3bfb6 (2,)
0x99722d376145463eca4e71c423a4dcc8e9dbda13 (47,)
0x7e7795e5e3fd2a9e9911d2b35e3ccbfe2917ecb0 (1,)
0x603f3fcd4c738c3f182f9379b448bde842370964 (0,)
0x98faf4aaa59a55687a1f65b0940b83e505528af7 (0,)
0x8477cd6ec6af9770b352a31cae45f73360c812e1 (0,)
0x62db875fd1205c07ef42efaa78685c0e37f4245e (0,)
0xb762bf9932aa3d21c818b2b7067c7c112fcd99ca (0,)
0x43665332e1a586e78068ad4004d99bfcf5ff5608 (2,)
0x1fd15ec8f3ab9a1f23362dfd2695e89e6

In [55]:
duration = 700000
depth = 0
def from_addr(addr,depth,st_time):
    depth +=1
    tx_addr = df[df['from'] == addr]
    for i in range(len(tx_addr)):
        if tx_addr['timestamp'].iloc[i] < (st_time+duration): 
            new_addr = tx_addr['to'].iloc[i]
            from_new_addr = df[df['from'] == new_addr]
            if len(from_new_addr) > 0:
                print((len(from_new_addr),depth))
                if depth < 3:
                    from_addr(new_addr,depth,st_time)

      

In [56]:
df_addr = df[df['from'] =='0x21f74c6bbc1e3ab9f0205e12de3a9daa14351aed']
st_time = df_addr['timestamp'].iloc[0]
from_addr('0x21f74c6bbc1e3ab9f0205e12de3a9daa14351aed',depth,st_time)

(212, 1)
(1, 2)
(3, 3)
(2, 2)
(10980, 3)
(10980, 3)
(1, 2)
(1, 2)
(72, 3)
(10, 2)
(1312, 3)
(1312, 3)
(1312, 3)
(1312, 3)
(1312, 3)
(1312, 3)
(1312, 3)
(208, 3)
(208, 3)
(1, 2)
(1, 3)
(39, 2)
(2, 3)
(212, 3)
(212, 3)
(1, 3)
(80, 3)
(2, 3)
(15, 3)
(2, 3)
(3, 3)
(6, 3)
(1, 3)
(5, 2)
(2, 3)
(5, 3)
(302, 3)
(22, 3)
(1, 2)
(4405, 3)
(2, 2)
(2, 3)
(2, 3)
(1, 2)
(1, 2)
(4, 2)
(11796, 3)
(11796, 3)
(11796, 3)
(11796, 3)
(4, 2)
(75, 3)
(2, 3)
(2, 2)
(4205, 3)
(4205, 3)
(1, 2)
(1, 2)
(1, 2)
(1, 2)
(7, 2)
(52765, 3)
(52765, 3)
(52765, 3)
(52765, 3)
(52765, 3)
(26, 3)
(52765, 3)
(1, 2)
(65671, 3)
(10, 2)
(6, 3)
(10, 3)
(6, 3)
(1, 2)
(3, 3)
(4, 2)
(3658, 3)
(3658, 3)
(3658, 3)
(3658, 3)
(1, 2)
(16, 3)
(1, 2)
(16, 3)
(3, 2)
(1, 2)
(159, 3)
(1, 2)
(16, 3)
(1, 2)
(3, 3)
(1, 2)
(159, 3)
(3, 2)
(2427, 3)
(2427, 3)
(2427, 3)
(1, 2)
(1, 2)
(4205, 3)
(2, 2)
(1, 3)
(1, 2)
(2, 2)
(1, 2)
(3, 3)
(1, 2)
(4, 2)
(1, 2)
(1, 3)
(1, 2)
(2, 2)
(4, 2)
(1, 3)
(19, 3)
(1, 3)
(1, 2)
(6, 3)
(1, 2)
(2, 3)
(2, 2)
(2, 2)
(2,

KeyboardInterrupt: 

In [50]:
st_time

1484475646

In [63]:
def load_node_csv(df):
    from_node = df['from'].unique()
    to_node = df['to'].unique()
    nodes1 = pd.DataFrame({'node':from_node})
    to  = pd.Series(to_node)
    nodes1 = nodes1['node'].append(to)
    nodes2 = pd.DataFrame({'node': nodes1})
    mapping = nodes2.node.unique()
    mapping = LabelEncoder().fit_transform(mapping)
    return mapping

In [64]:
nodes = load_node_csv(df)

In [67]:
def load_edge_csv(df):
    src = df['from'].values
    dst = df['to'].values
    src = LabelEncoder.fit_transform(src)
    dst = LabelEncoder.fit_transform(dst)
    print(len(src), len(dst))
    edge_index = torch.tensor([src, dst])

    return edge_index

In [70]:
# edge_index = load_edge_csv(df)